<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FineTuneALBERTModelForCustomDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00


In [59]:
import pandas as pd
import torch
import tensorflow as tf
from transformers import AlbertTokenizer, AlbertForMaskedLM
from torch.utils.data import DataLoader

In [70]:
# Load the dataset
dataset = pd.read_csv("output.csv",names=['labels','input_ids'])

In [71]:
dataset

,labels,input_ids
0,sum,int [MASK] = x + y;
1,numbers,int[3] [MASK] = [1
2,animals,"string[LENGTH] [MASK] = [""cat"""
3,indexes,int[] [MASK] = [1
4,colors,"string[*] [MASK] = [""red"""
...,...,...
58,engineer,"Engineer [MASK] = new Engineer(""Alice"");"
59,engineerName,string [MASK] = engineer.getName();
60,str,string [MASK] = check string
61,personName,"string [MASK] = ""John Smith"";"


In [72]:
from transformers import AlbertTokenizer

# Load the ALBERT tokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

# Tokenize the masked statements
inputs = tokenizer(list(dataset["input_ids"]), return_tensors="pt", padding=True, truncation=True)

# Convert the target variable names to label IDs
labels = tokenizer(list(dataset["labels"]), return_tensors="pt", padding=True, truncation=True)["input_ids"]


In [81]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.inputs.items()}, self.labels[idx]

# Define a data loader for your dataset
batch_size = 32
data_loader = DataLoader(MyDataset(inputs, labels), batch_size=batch_size, shuffle=True, drop_last=True)




In [82]:
from transformers import AlbertForMaskedLM, AdamW

# Load the pre-trained ALBERT model for masked language modeling
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

# Define the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=-100)

# Fine-tune the ALBERT model on your dataset
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = loss_fn(outputs.logits.view(-1, model.config.vocab_size), labels.view(-1))
        loss.backward()
        optimizer.step()


In [83]:
# Define a function to generate variable name suggestions for a masked statement
def suggest_variable_name(masked_statement):
    input_ids = tokenizer.encode(masked_statement, return_tensors="pt")
    outputs = model(input_ids)
    predictions = outputs.logits[0, input_ids[0].tolist().index(tokenizer.mask_token_id)].softmax(dim=-1)
    predicted_label_ids = predictions.argmax(dim=-1)
    predicted_label = tokenizer.decode(predicted_label_ids)
    return predicted_label


In [86]:
# Define a new masked statement for which you want to generate a variable name suggestion
masked_statement = "String [MASK] = getNames();"

# Generate a variable name suggestion using the suggest_variable_name function
variable_name = suggest_variable_name(masked_statement)

# Print the suggested variable name
print("The suggested variable name for the statement '{}' is '{}'.".format(masked_statement, variable_name))


The suggested variable name for the statement 'String [MASK] = getNames();' is 'parentheses'.
